In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sqrt
import xgboost as xgb

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV


%matplotlib inline
import time

In [2]:
train_df = pd.read_csv("../features/train.csv", low_memory=False)
test_df = pd.read_csv("../features/test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [3]:
train_df = train_df[['id','precio']]
#train_df['precio'].value_counts()

In [4]:
print(data.shape)
print(test_df.shape)
print(train_df.shape)

(300000, 215)
(60000, 22)
(240000, 2)


In [5]:
features = pd.merge(train_df, data, on='id', how='inner')

In [6]:
features.not_outlier.value_counts()

1.0    215351
0.0     24649
Name: not_outlier, dtype: int64

In [7]:
filter_name = 'not_outlier'
features = features[features[filter_name] == True]
features.shape
#features.not_outlier.value_counts()

(215351, 216)

In [8]:
features = features.fillna(0)

In [9]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
features = features.drop(['desc_comedor', 'desc_frente', 'desc_transport', 'desc_amoblado', 'titulo_escuela', 'desc_ambientes', 'titulo_avenida', 'desc_vende', 'titulo_zona', 'titulo_alquila', 'desc_auto', 'titulo_frente', 'desc_alquila', 'desc_anhelo', 'desc_zona', 'dia', 'mes', 'avenida', 'desc_avenida', 'desc_amenities_lujo', 'titulo_positiva', 'avenue_en_desc', 'titulo_anhelo', 'titulo_vende', 'titulo_amoblado', 'titulo_terminado', 'titulo_antiguedad', 'titulo_amenities_lujo', 'Desconocida', 'titulo_gym', 'titulo_auto', 'titulo_pago',  'titulo_comedor', 'titulo_estudio', 'titulo_pileta', 'titulo_transport', 'idzona_0'], axis=1)



In [10]:
features.shape

(215351, 177)

In [15]:
features = features.fillna(0)

In [16]:
from xgboost import XGBRegressor

cv = StratifiedKFold(5)
rfe = RFECV(XGBRegressor(), cv=cv, scoring='neg_mean_squared_error')



In [ ]:
rfe = rfe.fit(features, labels)

print(rfe.support_)
print(rfe.ranking_)

In [ ]:
mask = rfe.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, features):
    if bool:
        new_features.append(feature)

In [ ]:
features = features[new_features]

In [ ]:
regressor = XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.04,
                             gamma=1, subsample=0.75, colsample_bytree=0.75, max_depth=10)

regressor.fit(features, labels)

In [ ]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [ ]:
set_test = set_test.fillna(0)

In [ ]:
set_test = set_test[new_features]

In [ ]:
set_test.head()

In [ ]:
preds_kaggle = regressor.predict(set_test)

In [ ]:
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.head()

In [ ]:
df_kaggel = pd.DataFrame(df_kaggel)                               
df_kaggel["id"] = ids
df_kaggel.rename(columns = {0: 'target'},inplace = True) 
df_kaggel = df_kaggel[['id','target']]

In [ ]:
df_kaggel.head()

In [ ]:
df_kaggel.to_csv('ResultadosXGBrfe_ROB.csv',index=False)

In [ ]:
df_kaggel.shape

In [ ]:
df_kaggel.head()